In [ ]:
import fpdf
import pandas as pd
import xlsxwriter as xl
import math

In [ ]:
raw_data = pd.read_excel('./Budget_Comparison_p5000108_Accrual.xlsx')
raw_data

In [ ]:
raw_bug = pd.read_excel('./Budget_Comparison_p1002895_Accrual.xlsx')
raw_bug

In [ ]:
raw_bug_2 = pd.read_excel('./bug_to_fix_2.xlsx')
raw_bug_2

In [ ]:
income_statement_data_raw =  pd.read_excel('./income_statement_1.xlsx')
income_statement_data_raw

In [ ]:
def create_excel(df, xlfile, income_statement_1):
    yellow_color = '#b4992d'
    dark_gray_color = '#505050'
    white_color = '#FFFFFF'
    black_color = '#000000'
    grey_color = '#211f20'
    prop_name = df.columns[0]
    prop_name_is = df.columns[0]
    try:
        prop_name = prop_name.split('(', 1)[0]
    except:
        pass
    try:
        prop_name_is = prop_name_is('(', 1)[0]
    except:
        pass
    df = df.rename(columns={df.columns[0]: 'Col1'
                           , df.columns[1]: 'Col2'
                           , df.columns[2]: 'Col3'
                           , df.columns[3]: 'Col4'
                           , df.columns[4]: 'Col5'
                           , df.columns[5]: 'Col6'
                           , df.columns[6]: 'Col7'
                           , df.columns[7]: 'Col8'
                           , df.columns[8]: 'Col9'
                           })
    # income statement 1
    income_statement_1 = income_statement_1.rename(columns={income_statement_1.columns[0]: 'Col1'
                           , income_statement_1.columns[1]: 'Col2'
                           , income_statement_1.columns[2]: 'Col3'
                           , income_statement_1.columns[3]: 'Col4'
                           , income_statement_1.columns[4]: 'Col5'
                           })
    header_2 = df['Col1'][0]
    header_3 = df['Col1'][1]
    header_4 = df['Col1'][2]
    header_cols = ['MTD Actual'
                   ,'MTD Budget'
                   ,'Variance'
                   ,'% Var'
                   ,'YTD Actual'
                   ,'YTD Budget'
                   ,'Variance'
                   ,'% Var'
                   , 'Variance Explanations (5% and $2,500)']
    header_cols_is = ['Month to Date'
                  ,'Year to Date']
    header_2_is = income_statement_1['Col1'][0]
    header_3_is = income_statement_1['Col1'][1]
    header_4_is = income_statement_1['Col1'][2]
    # clean data
    df=df.dropna(subset=['Col1']).reset_index(drop=True)
    income_statement_1=income_statement_1.dropna(subset=['Col1']).reset_index(drop=True)
    def clean_text_col_1(text_val):
        return_text_val = ''
        index_val = 0
        for i in text_val:
            if index_val > 2:
                return_text_val = return_text_val + str(i)
            index_val += 1
        return return_text_val
    df['Col1'] = df.apply(lambda x: clean_text_col_1(x['Col1']), axis=1)
    income_statement_1['Col1'] = income_statement_1.apply(lambda x: clean_text_col_1(x['Col1']), axis=1)
    def flag_zero_vals(val_1, val_2):
        flag_zero_val = 0
        if(math.isnan(val_1)):
            if(math.isnan(val_2)):
                flag_zero_val = 1
        return flag_zero_val
    def flag_zero_vals_2(val_1, val_2):
        flag_zero_val = 0
        if(val_1 == 0):
            if(val_2 == 0):
                flag_zero_val = 1
        return flag_zero_val
    df['Nan_Var_Check'] = df.apply(lambda x: flag_zero_vals(x['Col5'], x['Col9']), axis=1)
    income_statement_1['Nan_Var_Check'] = income_statement_1.apply(lambda x: flag_zero_vals_2(x['Col2'], x['Col4']), axis=1)
    def flag_total_rows(val_1):
        flag_total_val = 0
        if 'TOTAL' in val_1:
            flag_total_val = 1
        return flag_total_val
    df['Total_Check'] = df.apply(lambda x: flag_total_rows(x['Col1']), axis=1)
    income_statement_1['Total_Check'] = income_statement_1.apply(lambda x: flag_total_rows(x['Col1']), axis=1)
    def flag_header_rows(val_1):
        flag_total_val = 0
        if val_1[0] != ' ':
            flag_total_val = 1
        return flag_total_val
    df['Header_Check'] = df.apply(lambda x: flag_header_rows(x['Col1']), axis=1)
    income_statement_1['Header_Check'] = income_statement_1.apply(lambda x: flag_header_rows(x['Col1']), axis=1)
    # wirte excel
    workbook = xl.Workbook(xlfile)
    worksheet = workbook.add_worksheet('Budget Comparison')
    header_format_1 = workbook.add_format({'font_color': black_color
                                    , 'bold':True
                                    , 'font_name': 'Century Gothic'
                                    , 'font_size':14
                                    , 'align':'left'
                                     })
    worksheet.merge_range("A1:I1", prop_name, header_format_1)
    worksheet.merge_range("A2:I2", header_2, header_format_1)
    header_format_2 = workbook.add_format({'font_color': dark_gray_color
                                    , 'bold':False
                                    , 'font_name': 'Century Gothic'
                                    , 'font_size':10
                                    , 'align':'left'
                                     })
    worksheet.merge_range("A3:I3", header_3, header_format_2)
    worksheet.merge_range("A4:I4", header_4, header_format_2)
    header_format_3 = workbook.add_format({'font_color': white_color
                                    , 'bg_color':black_color
                                    , 'bold':True
                                    , 'font_name': 'Century Gothic'
                                    , 'font_size':11
                                    , 'align':'center'
                                     })
    for row in range(10):
        if row == 0:
            worksheet.write_blank(4, row, '', header_format_3)
        else:
            worksheet.write_string(4, row, header_cols[row - 1], header_format_3)
    worksheet.merge_range(5, 0, 5, 9, '', header_format_2)
    worksheet.set_row(5,7.5)
    row_write_val = 6
    row_val_format_header = workbook.add_format({'font_color': black_color
                                    #, 'bg_color':black_color
                                    , 'bold':True
                                    , 'font_name': 'Century Gothic'
                                    , 'font_size':10
                                    , 'align':'left'
                                     })
    row_val_format_sub_item = workbook.add_format({'font_color': dark_gray_color
                                    #, 'bg_color':black_color
                                    , 'bold':True
                                    , 'font_name': 'Century Gothic'
                                    , 'font_size':10
                                    , 'align':'left'
                                     })
    row_val_format_sub_item_num = workbook.add_format({'font_color': dark_gray_color
                                    #, 'bg_color':black_color
                                    , 'bold':False
                                    , 'font_name': 'Century Gothic'
                                    , 'font_size':10
                                    , 'align':'right'
                                    , 'num_format':44
                                     })
    row_val_format_sub_item_percent = workbook.add_format({'font_color': dark_gray_color
                                    #, 'bg_color':black_color
                                    , 'bold':False
                                    , 'font_name': 'Century Gothic'
                                    , 'font_size':10
                                    , 'align':'right'
                                    , 'num_format':'#,##0.00%;[Red](#,##0.00%)'
                                     })
    row_val_format_total_item_num = workbook.add_format({'font_color': black_color
                                    #, 'bg_color':black_color
                                    , 'bold':True
                                    , 'font_name': 'Century Gothic'
                                    , 'font_size':10
                                    , 'align':'right'
                                    , 'num_format':'_($* #,##0.00_);[Red]_($* (#,##0.00);_($* "-"??_);_(@_)'
                                    , 'top':1
                                    , 'border_color':black_color
                                    })
    row_val_format_total_item_percent = workbook.add_format({'font_color': black_color
                                    #, 'bg_color':black_color
                                    , 'bold':True
                                    , 'font_name': 'Century Gothic'
                                    , 'font_size':10
                                    , 'align':'right'
                                    , 'num_format':'#,##0.00%;[Red](#,##0.00%)'
                                    , 'top':1
                                    , 'border_color':black_color
                                     })
    #-------------------------------------------------------------------------
    for i in range(4, df.shape[0] - 1):
        if df['Header_Check'][i] == 1:
            try:
                next_header_val = df['Header_Check'][i+1]
                next_total_val = df['Total_Check'][i+1]
                next_nan_val = df['Nan_Var_Check'][i+1]
            except:
                next_header_val = 0
                next_total_val = 0
                next_nan_val = 0
            # try to get next vals for logic
            new_row_needed = 0
            if df['Col1'][i] in ['OPERATING INCOME', 'OPERATING EXPENSES', 'RECOVERABLE', 'NON-RECOVERABLE']:
                worksheet.write_string(row_write_val, 0, df['Col1'][i], row_val_format_header)
                row_write_val = row_write_val + 1
                worksheet.merge_range(row_write_val, 0, row_write_val, 9, '', header_format_2)
                worksheet.set_row(row_write_val,7.5)
                row_write_val = row_write_val + 1
            else:
                if df['Total_Check'][i] == 1:
                    if next_total_val == 1:
                        worksheet.write_string(row_write_val, 0, df['Col1'][i], row_val_format_header)
                        worksheet.write_number(row_write_val, 1, df['Col2'][i],row_val_format_total_item_num)
                        worksheet.write_number(row_write_val, 2, df['Col3'][i],row_val_format_total_item_num)
                        worksheet.write_number(row_write_val, 3, df['Col4'][i],row_val_format_total_item_num)
                        try:
                            worksheet.write_number(row_write_val, 4, df['Col5'][i]/100,row_val_format_total_item_percent)
                        except:
                            worksheet.write_number(row_write_val, 4, 0,row_val_format_total_item_percent)
                        worksheet.write_number(row_write_val, 5, df['Col6'][i],row_val_format_total_item_num)
                        worksheet.write_number(row_write_val, 6, df['Col7'][i],row_val_format_total_item_num)
                        worksheet.write_number(row_write_val, 7, df['Col8'][i],row_val_format_total_item_num)
                        try:
                            worksheet.write_number(row_write_val, 8, df['Col9'][i]/100,row_val_format_total_item_percent)
                        except:
                            worksheet.write_number(row_write_val, 8, 0,row_val_format_total_item_percent)
                        row_write_val = row_write_val + 1
                    else:
                        if df['Nan_Var_Check'][i] == 0:
                            worksheet.write_string(row_write_val, 0, df['Col1'][i], row_val_format_header)
                            worksheet.write_number(row_write_val, 1, df['Col2'][i],row_val_format_total_item_num)
                            worksheet.write_number(row_write_val, 2, df['Col3'][i],row_val_format_total_item_num)
                            worksheet.write_number(row_write_val, 3, df['Col4'][i],row_val_format_total_item_num)
                            try:
                                worksheet.write_number(row_write_val, 4, df['Col5'][i]/100,row_val_format_total_item_percent)
                            except:
                                worksheet.write_number(row_write_val, 4, 0,row_val_format_total_item_percent)
                            worksheet.write_number(row_write_val, 5, df['Col6'][i],row_val_format_total_item_num)
                            worksheet.write_number(row_write_val, 6, df['Col7'][i],row_val_format_total_item_num)
                            worksheet.write_number(row_write_val, 7, df['Col8'][i],row_val_format_total_item_num)
                            try:
                                worksheet.write_number(row_write_val, 8, df['Col9'][i]/100,row_val_format_total_item_percent)
                            except:
                                worksheet.write_number(row_write_val, 8, 0,row_val_format_total_item_percent)
                            row_write_val = row_write_val + 1
                            if next_header_val == 1:
                                new_row_needed = 1
                            else:
                                pass
                else:
                    if next_nan_val == 0:
                        worksheet.write_string(row_write_val, 0, df['Col1'][i], row_val_format_header)
                        row_write_val = row_write_val + 1
                    elif next_header_val == 0:
                        row_val_check = i
                        value_needed = 0
                        next_header_found = 0
                        while row_val_check <= df.shape[0] - 1 and next_header_found == 1:
                            if df['Header_Check'][row_val_check] == 1:
                                next_header_found = 1
                            elif df['Nan_Var_Check'][row_val_check] == 1:
                                value_needed = 1
                            else:
                                row_val_check = row_val_check + 1
                        if value_needed == 1:
                            worksheet.write_string(row_write_val, 0, df['Col1'][i], row_val_format_header)
                            row_write_val = row_write_val + 1
                    else:
                        pass
            # ------------------------------------------
            # add a row or not
            if new_row_needed == 1:
                worksheet.merge_range(row_write_val, 0, row_write_val, 9, '', header_format_2)
                worksheet.set_row(row_write_val,7.5)
                row_write_val = row_write_val + 1
            else:
                pass
        else:
            if df['Nan_Var_Check'][i] == 1:
                pass
            else:
                worksheet.write_string(row_write_val, 0, df['Col1'][i],row_val_format_sub_item)
                worksheet.write_number(row_write_val, 1, df['Col2'][i],row_val_format_sub_item_num)
                worksheet.write_number(row_write_val, 2, df['Col3'][i],row_val_format_sub_item_num)
                worksheet.write_number(row_write_val, 3, df['Col4'][i],row_val_format_sub_item_num)
                try:
                    worksheet.write_number(row_write_val, 4, df['Col5'][i]/100,row_val_format_sub_item_percent)
                except:
                    worksheet.write_number(row_write_val, 4, 0,row_val_format_total_item_percent)
                worksheet.write_number(row_write_val, 5, df['Col6'][i],row_val_format_sub_item_num)
                worksheet.write_number(row_write_val, 6, df['Col7'][i],row_val_format_sub_item_num)
                worksheet.write_number(row_write_val, 7, df['Col8'][i],row_val_format_sub_item_num)
                try:
                    worksheet.write_number(row_write_val, 8, df['Col9'][i]/100,row_val_format_sub_item_percent)
                except:
                    worksheet.write_number(row_write_val, 8, 0,row_val_format_total_item_percent)
                row_write_val = row_write_val + 1
    worksheet.set_column(0,0,49.29)
    worksheet.set_column(9,9,49.29)
    worksheet.set_column(1,8,15)
    worksheet.print_area(0,0, row_write_val - 1, 9)
    num_pages_budget_1 = math.ceil(row_write_val/65)
    worksheet.fit_to_pages(1, num_pages_budget_1)
    worksheet.set_landscape()
    #-------------------------------------------------------------------------
    #-------------------------------------------------------------------------
    #-------------------------------------------------------------------------
    #-------------------------------------------------------------------------
    #-------------------------------------------------------------------------
    #-------------------------------------------------------------------------
    #-------------------------------------------------------------------------
    #-------------------------------------------------------------------------
    Income_Statement = workbook.add_worksheet('Income Statement')
    Income_Statement.merge_range("A1:C1", prop_name_is, header_format_1)
    Income_Statement.merge_range("A2:C2", header_2_is, header_format_1)
    Income_Statement.merge_range("A3:C3", header_3_is, header_format_2)
    Income_Statement.merge_range("A4:C4", header_4_is, header_format_2)
    for row in range(3):
        if row == 0:
            Income_Statement.write_blank(4, row, '', header_format_3)
            Income_Statement.write_blank(5, row, '', header_format_3)
        else:
            Income_Statement.write_string(4, row, header_cols_is[row - 1], header_format_3)
    Income_Statement.write_formula(5, 1, '=+TEXT(RIGHT(A3,8),"mmmm yyyy")', header_format_3)
    Income_Statement.write_formula(5, 2, '=+B6', header_format_3)
    Income_Statement.merge_range(6, 0, 6, 9, '', header_format_2)
    Income_Statement.set_row(6,7.5)
    row_write_val = 7
    for i in range(4, income_statement_1.shape[0] - 1):
        if income_statement_1['Header_Check'][i] == 1:
            try:
                next_header_val = income_statement_1['Header_Check'][i+1]
                next_total_val = income_statement_1['Total_Check'][i+1]
                next_nan_val = income_statement_1['Nan_Var_Check'][i+1]
            except:
                next_header_val = 0
                next_total_val = 0
                next_nan_val = 0
            # try to get next vals for logic
            new_row_needed = 0
            if income_statement_1['Col1'][i] in ['OPERATING INCOME', 'OPERATING EXPENSES', 'RECOVERABLE', 'NON-RECOVERABLE']:
                Income_Statement.write_string(row_write_val, 0, income_statement_1['Col1'][i], row_val_format_header)
                row_write_val = row_write_val + 1
                Income_Statement.merge_range(row_write_val, 0, row_write_val, 9, '', header_format_2)
                Income_Statement.set_row(row_write_val,7.5)
                row_write_val = row_write_val + 1
            else:
                if income_statement_1['Total_Check'][i] == 1:
                    if next_total_val == 1:
                        Income_Statement.write_string(row_write_val, 0, income_statement_1['Col1'][i], row_val_format_header)
                        Income_Statement.write_number(row_write_val, 1, income_statement_1['Col2'][i],row_val_format_total_item_num)
                        Income_Statement.write_number(row_write_val, 2, income_statement_1['Col4'][i],row_val_format_total_item_num)
                        row_write_val = row_write_val + 1
                    else:
                        if income_statement_1['Nan_Var_Check'][i] == 0:
                            Income_Statement.write_string(row_write_val, 0, income_statement_1['Col1'][i], row_val_format_header)
                            Income_Statement.write_number(row_write_val, 1, income_statement_1['Col2'][i],row_val_format_total_item_num)
                            Income_Statement.write_number(row_write_val, 2, income_statement_1['Col4'][i],row_val_format_total_item_num)
                            row_write_val = row_write_val + 1
                            if next_header_val == 1:
                                new_row_needed = 1
                            else:
                                pass
                else:
                    if next_nan_val == 0:
                        Income_Statement.write_string(row_write_val, 0, income_statement_1['Col1'][i], row_val_format_header)
                        row_write_val = row_write_val + 1
                    elif next_header_val == 0:
                        row_val_check = i
                        value_needed = 0
                        next_header_found = 0
                        while row_val_check <= income_statement_1.shape[0] - 1 and next_header_found == 1:
                            if income_statement_1['Header_Check'][row_val_check] == 1:
                                next_header_found = 1
                            elif income_statement_1['Nan_Var_Check'][row_val_check] == 1:
                                value_needed = 1
                            else:
                                row_val_check = row_val_check + 1
                        if value_needed == 1:
                            Income_Statement.write_string(row_write_val, 0, income_statement_1['Col1'][i], row_val_format_header)
                            row_write_val = row_write_val + 1
                    else:
                        pass
            # ------------------------------------------
            # add a row or not
            if new_row_needed == 1:
                Income_Statement.merge_range(row_write_val, 0, row_write_val, 9, '', header_format_2)
                Income_Statement.set_row(row_write_val,7.5)
                row_write_val = row_write_val + 1
            else:
                pass
        else:
            if income_statement_1['Nan_Var_Check'][i] == 1:
                pass
            else:
                Income_Statement.write_string(row_write_val, 0, income_statement_1['Col1'][i],row_val_format_sub_item)
                Income_Statement.write_number(row_write_val, 1, income_statement_1['Col2'][i],row_val_format_sub_item_num)
                Income_Statement.write_number(row_write_val, 2, income_statement_1['Col4'][i],row_val_format_sub_item_num)
                row_write_val = row_write_val + 1
    workbook.close()
    return income_statement_1

In [ ]:
create_excel(raw_bug_2, 'test_bug_fix_3.xlsx', income_statement_data_raw)

In [ ]:
create_excel(raw_bug, 'test_bug_fix_2.xlsx', income_statement_data_raw)

In [ ]:
create_excel(raw_data, 'Budget_Test.xlsx', income_statement_data_raw)

In [ ]:
b = create_excel(raw_data, 'Budget_Test.xlsx')
try:
    for i in range(b.shape[0]):
        print(b['Col1'][i])
        print(b['Nan_Var_Check'][i])
        print(b['Header_Check'][i])
        print('-----------------')
except:
    pass

In [ ]:
test_str = '   OPERATING INCOME'
def clean_text_col_1(text_val):
    return_text_val = ''
    index_val = 0
    for i in text_val:
        if index_val > 2:
            return_text_val = return_text_val + str(i)
        index_val += 1
    return return_text_val
clean_text_col_1(test_str)

In [ ]:
math.ceil(230/60)

In [ ]:
a = create_excel(raw_data)

In [ ]:
a

In [ ]:
a['Col1'][5]

In [ ]:
def flag_zero_vals(val_1, val_2):
    flag_zero_val = 0
    if(math.isnan(val_1)):
        if(math.isnan(val_2)):
            flag_zero_val = 1
    return flag_zero_val
flag_zero_vals(a['Col5'][0], a['Col9'][0])

In [ ]:
def flag_total_rows(val_1):
    flag_total_val = 0
    if 'TOTAL' in val_1:
        if val_1[0] != ' ':
            flag_total_val = 1
    return flag_total_val
test_1 = flag_total_rows(a['Col1'][0])
test_2 = flag_total_rows(a['Col1'][196])
print(test_1)
print(test_2)